# Preparing existing dataframes to be used in R for topic analysis

In [6]:
# import libraries
import os
import pickle
import pandas as pd
import re

In [7]:
# set up working directory
os.path.abspath(os.getcwd()) # initial working directory (should be equal to source file directory if using Jupyter Notebook)
os.chdir('../../data/web_scraping') # change to directory where all data files are stored
# check working directory
os.path.abspath(os.getcwd())

'/Users/patrickschulze/Desktop/Consulting/Bundestag-MP-Analyse/data/web_scraping'

## 1. se_df

In [102]:
with open('se_df.pickle', 'rb') as handle:
    se_df = pickle.load(handle)

In [113]:
# save as csv, not dropping any columns
se_df.to_csv('se_df.csv', index = False)

## 2. abg_df

In [104]:
with open('abg_df.pickle', 'rb') as handle:
    abg_df = pickle.load(handle)

##### Soziale Medien

In [105]:
# dropping column "Soziale Medien"
abg_df = abg_df.drop('Soziale Medien', axis = 1)

#### Ausschüsse

In [106]:
# # get all keys of the "Ausschuesse"-dicts
# set().union(*(d.keys() for d in abg_df['Ausschuesse']))

In [107]:
def get_ausschuss_positions(ausschuss_dict):
    ausschuesse_list = []
    if 'Ordentliches Mitglied' in ausschuss_dict:
        ausschuesse_list.append(ausschuss_dict['Ordentliches Mitglied'])
    if 'Stellvertretendes Mitglied' in ausschuss_dict:
        ausschuesse_list.append(ausschuss_dict['Stellvertretendes Mitglied'])
    if 'Obmann' in ausschuss_dict:
        ausschuesse_list.append(ausschuss_dict['Obmann'])
    if 'Obfrau' in ausschuss_dict:
        ausschuesse_list.append(ausschuss_dict['Obfrau'])
    if 'Stellv. Vorsitzender' in ausschuss_dict:
        ausschuesse_list.append(ausschuss_dict['Stellv. Vorsitzender'])
    if 'Stellv. Vorsitzende' in ausschuss_dict:
        ausschuesse_list.append(ausschuss_dict['Stellv. Vorsitzende'])
    if 'Vorsitzender' in ausschuss_dict:
        ausschuesse_list.append(ausschuss_dict['Vorsitzender'])
    if 'Vorsitzende' in ausschuss_dict:
        ausschuesse_list.append(ausschuss_dict['Vorsitzende'])
    
    positions = [item for sublist in ausschuesse_list for item in sublist] # flattening the list
    return positions

In [108]:
# creating a single column with all Ausschusse where the MdB is Mitglied, Obmann-/frau, or Vorsitz
abg_df['Ausschusspositionen'] = abg_df['Ausschuesse'].apply(get_ausschuss_positions)

#### Biografie

In [109]:
def get_year_of_birth(biografie):
    separators = re.compile('[,;]')
    sentence = separators.split(biografie)[0]
    sentence_stripped = " ".join(sentence.split('.')) # strip "." from sentence
    words = sentence_stripped.split(' ')
    four_digit_numbers = []
    for word in words:
        if len(word) == 4 and word.isdigit():
            four_digit_numbers.append(int(word))
    if len(four_digit_numbers) == 1:
        return four_digit_numbers[0]
    else:
        best_guess = min(four_digit_numbers)
        return(best_guess)

In [110]:
# extracting year of birth from biography
abg_df['Geburtsjahr'] = abg_df['Biografie'].apply(get_year_of_birth)

In [111]:
abg_df.head()

,Name,Partei,Wahlart,Bundesland,Wahlkreis,Wahlkreis-Nr.,Ausschuesse,Biografie,Twitter,Ausschusspositionen,Geburtsjahr
0,"Abercron, Dr. Michael von",CDU/CSU,Direkt gewählt,Schleswig-Holstein,Pinneberg,7,{'Ordentliches Mitglied': ['Ausschuss für Ernä...,Geboren am 17. November 1952 in Ehlers...,mvabercron,"[Ausschuss für Ernährung und Landwirtschaft, A...",1952
1,"Achelwilm, Doris",Die Linke,Gewählt über Landesliste,Bremen,,,{'Ordentliches Mitglied': ['Ausschuss für Fami...,Geboren am 30. November 1976 in Thuine...,DorisAchelwilm,"[Ausschuss für Familie, Senioren, Frauen und J...",1976
2,"Aggelidis, Grigorios",FDP,Gewählt über Landesliste,Niedersachsen,Hannover-Land I,43,{'Ordentliches Mitglied': ['Kuratorium der Bun...,Geboren am 19. August 1965 in Hannover...,aggelidis_fdp,[Kuratorium der Bundeszentrale für politische ...,1965
3,"Akbulut, Gökay",Die Linke,Gewählt über Landesliste,Baden-Württemberg,Mannheim,275,"{'Ordentliches Mitglied': ['Schriftführer/in',...",Geboren 1982 in Pinarbasi/ Türkei; ledig.Juni ...,akbulutgokay,"[Schriftführer/in, Ausschuss für Recht und Ver...",1982
4,"Albani, Stephan",CDU/CSU,Gewählt über Landesliste,Niedersachsen,Oldenburg – Ammerland,27,{'Ordentliches Mitglied': ['Ausschuss für Bild...,Geboren am 3. Juni 1968 in Göttingen; verheira...,,"[Ausschuss für Bildung, Forschung und Technikf...",1968


##### Saving as CSV

In [114]:
abg_df.to_csv('abg_df.csv', index = False)

## 3. tweepy_df

We initially stored tweepy output as pickle file (compressed file format). To use it within R, however, we have to convert to csv. Note that this last piece of code should be run on a local machine because the resulting dataset, tweepy_df.csv, is very large (~5 GB). Therefore, only the pickle file is directly available in the data folder.

In [8]:
with open('tweepy_df.pickle', 'rb') as handle:
    tweepy_df = pickle.load(handle)

In [ ]:
tweepy_df.to_csv('tweepy_df.csv', index = False)